In [1]:
from powersimdata.scenario.scenario import Scenario
from powersimdata.design.scenario_info import ScenarioInfo
import json
import os
import pandas as pd
from collections import defaultdict
import warnings
from pytest import approx

In [2]:
from powersimdata.design.clean_capacity_scaling import CollaborativeStrategyManager,\
IndependentStrategyManager, TargetManager, ResourceManager, Resource

In [3]:
from powersimdata.input.grid import Grid
from powersimdata.input.change_table import ChangeTable
from powersimdata.utility.constants import abv2state, state2loadzone

In [4]:
eastern = Grid(['Eastern'])
pmax = eastern.plant.groupby(['zone_name','type'])['Pmax'].sum()
sum_by_type_zone = defaultdict(dict)
for index, value in pmax.items():
    sum_by_type_zone[index[0]][index[1]] = value
sum_by_type_zone

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv


defaultdict(dict,
            {'Alabama': {'coal': 6671.200999999997,
              'dfo': 47.801,
              'hydro': 3319.292000000001,
              'ng': 15382.995999999997,
              'nuclear': 4547.1089999999995,
              'other': 381.0,
              'solar': 154.2,
              'wind': 1.0},
             'Arkansas': {'coal': 5487.0,
              'dfo': 12.299999999999997,
              'hydro': 1348.9020000000003,
              'ng': 7229.396999999999,
              'nuclear': 1529.424,
              'other': 156.97,
              'solar': 94.0,
              'wind': 1.0},
             'Chicago North Illinois': {'coal': 2774.7219999999998,
              'dfo': 477.238,
              'hydro': 39.69800000000001,
              'ng': 12169.88100000001,
              'nuclear': 7989.376,
              'other': 64.266,
              'solar': 31.457,
              'wind': 1315.5939999999998},
             'Connecticut': {'coal': 400.0,
              'dfo': 2816.807999999

In [5]:
gen_capacity_filename = 'Generator_Capacity_Collaborative_Anchor.xlsx'
gen_capacity = pd.read_excel(io=gen_capacity_filename)
gen_capacity['region_name'] = gen_capacity['region_name'].apply(lambda x: x.strip())
gen_capacity.set_index('region_name', inplace=True)
gen_capacity

,Coal,DFO,Geo-thermal,Hydro,Natural Gas,Nuclear,Other,Solar,Wind
region_name,,,,,,,,,
Alabama,5504.5,47.8,0,3319.3,15433.0,4547.110749,381.002460,439.869002,2.233972
Arkansas,5487.0,12.3,0,1348.9,7229.4,1529.423549,156.969505,245.736872,2.233972
Connecticut,400.0,2816.8,0,149.5,5331.5,1888.898461,166.000000,294.884247,11.169858
Delaware,445.5,121.6,0,0.0,2978.6,0.000000,10.326080,83.997331,4.467943
Florida,8768.0,3794.3,0,55.7,46472.7,3341.230803,887.000000,4788.294659,6.701915
Georgia,8979.1,1072.6,0,3597.8,18652.3,3929.323449,546.008206,3388.488074,4.467943
Illinois,13422.0,811.0,0,39.7,17600.0,11236.992685,84.348784,85.114317,11621.343494
Indiana,16981.9,115.5,0,92.1,9767.7,0.000000,91.000000,546.429446,5160.027528
Iowa,6034.5,1086.1,0,129.2,3724.8,535.903048,34.476138,27.477850,22129.275558


In [6]:
colname_map = {'Coal':'coal',
               'DFO':'dfo',
               'Geo-thermal':'geothermal',
               'Hydro':'hydro',
               'Natural Gas':'ng',
               'Nuclear':'nuclear',
               'Solar':'solar',
               'Wind':'wind',
               'Storage':'storage',
               'Biomass':'biomass',
               'Other': 'other'
               }

In [7]:
# Calculate scale factors
# Storage, Biomass, Geo-thermal are excluded (not scaled accordingly) for now
scale_factor = defaultdict(dict)
for area, row in gen_capacity.iterrows():
    if area not in state2loadzone and area not in eastern.zone2id:
        print('No Such State or Loadzone %s:'%row['region_name'])
        continue
    for gen_type in gen_capacity.columns:
        if gen_type not in {'region_name','Storage','Biomass','Geo-thermal'}:
            sum_state_ca = 0
            if area in state2loadzone:
                for loadzone in state2loadzone[area]:
                    if loadzone in sum_by_type_zone and colname_map[gen_type] in sum_by_type_zone[loadzone]:
                        sum_state_ca += sum_by_type_zone[loadzone][colname_map[gen_type]]
                for loadzone in state2loadzone[area]:
                    if loadzone in sum_by_type_zone and colname_map[gen_type] in sum_by_type_zone[loadzone]:
                        scale_factor[colname_map[gen_type]][loadzone] = (row[gen_type],sum_state_ca)
            else:
                if colname_map[gen_type] in sum_by_type_zone[area]:
                    sum_state_ca = sum_by_type_zone[area][colname_map[gen_type]]
                scale_factor[colname_map[gen_type]][area] = (row[gen_type],sum_state_ca)
scale_factor_input = defaultdict(dict)
for gen_type in scale_factor:
    for state in scale_factor[gen_type]:
        if scale_factor[gen_type][state][1] == 0 and scale_factor[gen_type][state][0] != 0:
            print('Error: Trying to scale over 0!',state,gen_type)
        elif scale_factor[gen_type][state][1] == 0:
            print(f"scale_factor zero: {gen_type} {state}")
            scale_factor_input[gen_type][state] = 0
        else:
            scale_factor_input[gen_type][state] = scale_factor[gen_type][state][0]/scale_factor[gen_type][state][1]
scale_factor_input

scale_factor zero: coal Maine
scale_factor zero: coal New Mexico Eastern
scale_factor zero: dfo Montana Eastern
scale_factor zero: dfo New Mexico Eastern
scale_factor zero: hydro Montana Eastern
scale_factor zero: hydro New Mexico Eastern
scale_factor zero: nuclear Montana Eastern
scale_factor zero: nuclear New Mexico Eastern


defaultdict(dict,
            {'coal': {'Alabama': 0.8251137988497131,
              'Arkansas': 1.0,
              'Connecticut': 1.0,
              'Delaware': 1.0,
              'Florida North': 0.7906010804400525,
              'Florida Panhandle': 0.7906010804400525,
              'Florida South': 0.7906010804400525,
              'Georgia South': 0.8881670736521573,
              'Georgia North': 0.8881670736521573,
              'Chicago North Illinois': 0.856940264840638,
              'Illinois Downstate': 0.856940264840638,
              'Indiana': 0.9656815390763852,
              'Iowa': 1.000000497141683,
              'Kansas': 0.9834228946882184,
              'Kentucky': 0.8650083003775229,
              'Louisiana': 1.0000000000000004,
              'Maine': 0,
              'Maryland': 0.9093388041902147,
              'Massachusetts': 0.0,
              'Michigan Southern': 0.9115149604638346,
              'Michigan Northern': 0.9115149604638346,
              'Minn

In [8]:
significant_scale_factor = defaultdict(dict)
for gen_type, next_level in scale_factor_input.items():
    for load_zone, scale_factor in next_level.items():
        if abs(scale_factor - 1.0) > 0.001:
            significant_scale_factor[gen_type][load_zone] = scale_factor
significant_scale_factor

defaultdict(dict,
            {'coal': {'Alabama': 0.8251137988497131,
              'Florida North': 0.7906010804400525,
              'Florida Panhandle': 0.7906010804400525,
              'Florida South': 0.7906010804400525,
              'Georgia South': 0.8881670736521573,
              'Georgia North': 0.8881670736521573,
              'Chicago North Illinois': 0.856940264840638,
              'Illinois Downstate': 0.856940264840638,
              'Indiana': 0.9656815390763852,
              'Kansas': 0.9834228946882184,
              'Kentucky': 0.8650083003775229,
              'Maine': 0,
              'Maryland': 0.9093388041902147,
              'Massachusetts': 0.0,
              'Michigan Southern': 0.9115149604638346,
              'Michigan Northern': 0.9115149604638346,
              'Minnesota Northern': 0.967782052826891,
              'Minnesota Southern': 0.967782052826891,
              'Mississippi': 0.8010239257584183,
              'Missouri West': 0.91913897191

In [9]:
capacities_df = gen_capacity.rename(columns=colname_map)

In [10]:
base_grid = Grid(['Eastern'])
independent_strategy_manager = IndependentStrategyManager()
scale_factors_new = independent_strategy_manager.create_scale_factor_table(base_grid, capacities_df)
scale_factors_new

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv


/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/scenario_info.py:122: UserWarning: No such type of generator in the area specified!
  warnings.warn('No such type of generator in the area specified!')
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Florida and resource hydro when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))


Base grid capacity is zero for loadzone Alabama and resource geothermal
Base grid capacity is zero for loadzone Arkansas and resource geothermal
Base grid capacity is zero for loadzone Connecticut and resource geothermal
Base grid capacity is zero for loadzone Delaware and resource geothermal
Base grid capacity is zero for loadzone Delaware and resource hydro
Base grid capacity is zero for loadzone Delaware and resource nuclear
Base grid capacity is zero for loadzone Florida North and resource geothermal
Base grid capacity is zero for loadzone Florida Panhandle and resource geothermal
Base grid capacity is zero for loadzone Florida South and resource geothermal


/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Florida and resource nuclear when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Georgia and resource nuclear when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Georgia and resource wind when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Illinois and resource hydro when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))


Base grid capacity is zero for loadzone Georgia South and resource geothermal
Base grid capacity is zero for loadzone Georgia North and resource geothermal
Base grid capacity is zero for loadzone Chicago North Illinois and resource geothermal
Base grid capacity is zero for loadzone Illinois Downstate and resource geothermal
Base grid capacity is zero for loadzone Indiana and resource geothermal
Base grid capacity is zero for loadzone Indiana and resource nuclear
Base grid capacity is zero for loadzone Iowa and resource geothermal
Base grid capacity is zero for loadzone Kansas and resource geothermal
Base grid capacity is zero for loadzone Kentucky and resource geothermal
Base grid capacity is zero for loadzone Kentucky and resource nuclear
Base grid capacity is zero for loadzone Louisiana and resource geothermal
Base grid capacity is zero for loadzone Maine and resource coal
Base grid capacity is zero for loadzone Maine and resource geothermal
Base grid capacity is zero for loadzone Ma

...

/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area North Carolina and resource nuclear when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Ohio and resource hydro when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Ohio and resource nuclear when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))


Base grid capacity is zero for loadzone Rhode Island and resource coal
Base grid capacity is zero for loadzone Rhode Island and resource geothermal
Base grid capacity is zero for loadzone Rhode Island and resource nuclear
Base grid capacity is zero for loadzone South Carolina and resource geothermal
Base grid capacity is zero for loadzone South Dakota and resource geothermal
Base grid capacity is zero for loadzone South Dakota and resource nuclear
Base grid capacity is zero for loadzone Tennessee and resource geothermal
Base grid capacity is zero for loadzone East Texas and resource dfo
Base grid capacity is zero for loadzone Texas Panhandle and resource dfo
Base grid capacity is zero for loadzone East Texas and resource geothermal
Base grid capacity is zero for loadzone Texas Panhandle and resource geothermal
Base grid capacity is zero for loadzone East Texas and resource nuclear
Base grid capacity is zero for loadzone Texas Panhandle and resource nuclear


/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Texas and resource hydro when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Virginia and resource nuclear when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Virginia and resource solar when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))


Base grid capacity is zero for loadzone Vermont and resource coal
Base grid capacity is zero for loadzone Vermont and resource geothermal
Base grid capacity is zero for loadzone Vermont and resource ng
Base grid capacity is zero for loadzone Vermont and resource nuclear
Base grid capacity is zero for loadzone Virginia Mountains and resource geothermal
Base grid capacity is zero for loadzone Virginia Tidewater and resource geothermal
Base grid capacity is zero for loadzone West Virginia and resource geothermal
Base grid capacity is zero for loadzone West Virginia and resource nuclear
Base grid capacity is zero for loadzone Wisconsin and resource geothermal


defaultdict(dict,
            {'coal': {'Alabama': 0.8251137988497128,
              'Florida North': 0.7906010804400528,
              'Florida Panhandle': 0.7906010804400528,
              'Florida South': 0.7906010804400528,
              'Georgia South': 0.8881670736521574,
              'Georgia North': 0.8881670736521574,
              'Chicago North Illinois': 0.856940264840638,
              'Illinois Downstate': 0.856940264840638,
              'Indiana': 0.9656815390763844,
              'Kansas': 0.9834228946882186,
              'Kentucky': 0.865008300377523,
              'Maryland': 0.9093388041902145,
              'Massachusetts': 0.0,
              'Michigan Southern': 0.911514960463835,
              'Michigan Northern': 0.911514960463835,
              'Minnesota Northern': 0.9677820528268913,
              'Minnesota Southern': 0.9677820528268913,
              'Mississippi': 0.8010239257584183,
              'Missouri West': 0.9191389719135257,
              'Misso

In [11]:
scale_factor_table = scale_factors_new
answer = significant_scale_factor

print("first-level:")
print(f"{len(scale_factor_table)} <==> {len(answer)}")
print()
for gen_type, next_level in scale_factor_table.items():
    print(f"second-level: {gen_type}")
    try:
        print(f"{len(next_level)} <==> {len(answer[gen_type])}")
    except KeyError as e:
            print(f"**** Missing {e}")
    print()
    
#     print("third-level:")
    print()
    for region_name, scale_factor in next_level.items():
        #print(f"{region_name}")
        try:
            assert scale_factor == approx(answer[gen_type][region_name])
            #print(f"{scale_factor} <==> {answer[gen_type][region_name]}")
        except KeyError as e:
            print(f"**** Missing {e}")
            print(f"unmatched scale factor: {scale_factor}")
            print()
        #print()
    print()

first-level:
6 <==> 7

second-level: coal
33 <==> 35



second-level: ng
38 <==> 38



second-level: solar
50 <==> 50



second-level: wind
51 <==> 51



second-level: dfo
26 <==> 28



second-level: nuclear
1 <==> 3





In [12]:
scale_factor_table = significant_scale_factor
answer = scale_factors_new

print("first-level:")
print(f"{len(scale_factor_table)} <==> {len(answer)}")
print()
for gen_type, next_level in scale_factor_table.items():
    print(f"second-level: {gen_type}")
    try:
        print(f"{len(next_level)} <==> {len(answer[gen_type])}")
    except KeyError as e:
            print(f"**** Missing {e}")
    print()
    
#     print("third-level:")
    print()
    for region_name, scale_factor in next_level.items():
        #print(f"{region_name}")
        try:
            assert scale_factor == approx(answer[gen_type][region_name])
            #print(f"{scale_factor} <==> {answer[gen_type][region_name]}")
        except KeyError as e:
            print(f"**** Missing {e}")
            print(f"unmatched scale factor: {scale_factor}")
            print()
        #print()
    print()

first-level:
7 <==> 6

second-level: coal
35 <==> 33


**** Missing 'Maine'
unmatched scale factor: 0

**** Missing 'New Mexico Eastern'
unmatched scale factor: 0


second-level: dfo
28 <==> 26


**** Missing 'Montana Eastern'
unmatched scale factor: 0

**** Missing 'New Mexico Eastern'
unmatched scale factor: 0


second-level: hydro
2 <==> 0


**** Missing 'Montana Eastern'
unmatched scale factor: 0

**** Missing 'New Mexico Eastern'
unmatched scale factor: 0


second-level: ng
38 <==> 38



second-level: nuclear
3 <==> 1


**** Missing 'Montana Eastern'
unmatched scale factor: 0

**** Missing 'New Mexico Eastern'
unmatched scale factor: 0


second-level: solar
50 <==> 50



second-level: wind
51 <==> 51





In [15]:
change_table_new = independent_strategy_manager.create_change_table(scale_factors_new, 'Eastern')
change_table_new.ct

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv


{'coal': {'zone_id': {24: 0.8251137988497128,
   22: 0.7906010804400528,
   21: 0.7906010804400528,
   23: 0.7906010804400528,
   20: 0.8881670736521574,
   19: 0.8881670736521574,
   34: 0.856940264840638,
   35: 0.856940264840638,
   33: 0.9656815390763844,
   48: 0.9834228946882186,
   27: 0.865008300377523,
   13: 0.9093388041902145,
   4: 0.0,
   32: 0.911514960463835,
   31: 0.911514960463835,
   37: 0.9677820528268913,
   38: 0.9677820528268913,
   25: 0.8010239257584183,
   41: 0.9191389719135257,
   40: 0.9191389719135257,
   9: 0.392540757835182,
   16: 0.966104223824092,
   17: 0.966104223824092,
   51: 0.9562768671984551,
   29: 0.7548696051642252,
   30: 0.7548696051642252,
   11: 0.7940446194340687,
   10: 0.7940446194340687,
   26: 0.8166491334954616,
   14: 0.7707470744632977,
   15: 0.7707470744632977,
   28: 0.9674929131293992,
   36: 0.7655943379099679}},
 'ng': {'zone_id': {24: 1.0032506021583834,
   6: 1.0204997663847009,
   12: 0.9875339831576153,
   22: 0.9684479

In [16]:
change_table_old = ChangeTable('Eastern')
for gen_type in significant_scale_factor:
    change_table_old.scale_plant_capacity(
        gen_type, 
        zone_name=significant_scale_factor[gen_type])
change_table_old.ct

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
No coal plants in New Mexico Eastern.
No dfo plants in Montana Eastern.
No dfo plants in New Mexico Eastern.
No hydro plants in New Mexico Eastern.
No nuclear plants in Montana Eastern.
No nuclear plants in New Mexico Eastern.


{'coal': {'zone_id': {24: 0.8251137988497131,
   22: 0.7906010804400525,
   21: 0.7906010804400525,
   23: 0.7906010804400525,
   20: 0.8881670736521573,
   19: 0.8881670736521573,
   34: 0.856940264840638,
   35: 0.856940264840638,
   33: 0.9656815390763852,
   48: 0.9834228946882184,
   27: 0.8650083003775229,
   1: 0,
   13: 0.9093388041902147,
   4: 0.0,
   32: 0.9115149604638346,
   31: 0.9115149604638346,
   37: 0.967782052826891,
   38: 0.967782052826891,
   25: 0.8010239257584183,
   41: 0.9191389719135257,
   40: 0.9191389719135257,
   9: 0.392540757835182,
   16: 0.9661042238240921,
   17: 0.9661042238240921,
   51: 0.9562768671984551,
   29: 0.7548696051642252,
   30: 0.7548696051642252,
   11: 0.7940446194340687,
   10: 0.7940446194340687,
   26: 0.8166491334954618,
   14: 0.7707470744632975,
   15: 0.7707470744632975,
   28: 0.9674929131293993,
   36: 0.7655943379099678}},
 'dfo': {'zone_id': {22: 0.6699796903080986,
   21: 0.6699796903080986,
   23: 0.6699796903080986,
  

In [36]:
scale_factor_table = change_table_new.ct
answer = change_table_old.ct

print("first-level:")
print(f"{len(scale_factor_table)} <==> {len(answer)}")
print()
for gen_type, next_level in scale_factor_table.items():
    print(f"second-level: {gen_type}")
    try:
        print(f"{len(next_level)} <==> {len(answer[gen_type])}")
    except KeyError as e:
            print(f"**** Missing {e}")
    print()
    
    for operation, next_level in next_level.items():
    
#       print("third-level:")
        print()
        for region_name, scale_factor in next_level.items():
            #print(f"{region_name}")
            try:
                assert scale_factor == approx(answer[gen_type][operation][region_name])
                #print(f"{scale_factor} <==> {answer[gen_type][operation][region_name]}")
            except KeyError as e:
                print(f"load_zone: {region_name}")
                print(f"**** Missing {e}")
                print(f"unmatched scale factor: {scale_factor}")
                print()
            #print()
        print()

first-level:
6 <==> 7

second-level: coal
1 <==> 1



second-level: ng
1 <==> 1



second-level: solar
1 <==> 1



second-level: wind
1 <==> 1



second-level: dfo
1 <==> 1



second-level: nuclear
1 <==> 1





In [29]:
scale_factor_table = change_table_old.ct
answer = change_table_new.ct

print("first-level:")
print(f"{len(scale_factor_table)} <==> {len(answer)}")
print()
for gen_type, next_level in scale_factor_table.items():
    print(f"second-level: {gen_type}")
    try:
        print(f"{len(next_level)} <==> {len(answer[gen_type])}")
    except KeyError as e:
            print(f"**** Missing {e}")
    print()
    
    for operation, next_level in next_level.items():
    
#       print("third-level:")
        print()
        for region_name, scale_factor in next_level.items():
            #print(f"{region_name}")
            try:
                assert scale_factor == approx(answer[gen_type][operation][region_name])
                #print(f"{scale_factor} <==> {answer[gen_type][operation][region_name]}")
            except KeyError as e:
                print(f"load_zone: {region_name}")
                print(f"**** Missing {e}")
                print(f"unmatched scale factor: {scale_factor}")
                print()
            #print()
        print()

first-level:
7 <==> 6

second-level: coal
1 <==> 1


load_zone: 1
**** Missing 1
unmatched scale factor: 0


second-level: dfo
1 <==> 1



second-level: hydro
**** Missing 'hydro'


load_zone: 52
**** Missing 'hydro'
unmatched scale factor: 0


second-level: ng
1 <==> 1



second-level: nuclear
1 <==> 1



second-level: solar
1 <==> 1



second-level: wind
1 <==> 1





In [31]:
eastern.id2zone[1]

'Maine'

In [35]:
sum_by_type_zone['Maine']['coal']

0.0

In [32]:
eastern.id2zone[52]

'Montana Eastern'

In [34]:
sum_by_type_zone['Montana Eastern']['hydro']

0.0

In [22]:
# scale_factor_table = change_table_new.ct
# answer = change_table_old.ct

# assert len(scale_factor_table) == len(answer)
# for gen_type, next_level in scale_factor_table.items():
#     assert len(next_level) == len(answer[gen_type])
#     for operation, next_level in next_level.items():
#         assert len(next_level) == len(answer[gen_type][operation])
#         for region_name, scale_factor in next_level.items():
#                 assert scale_factor == approx(answer[gen_type][operation][region_name])

AssertionError: 